In [1]:
# import packages
import pdb
import os
import pickle
from tqdm.notebook import tqdm
import random
from tabulate import tabulate

import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import torchvision.transforms.v2 as t

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from torchvision import models
from torchvision.transforms.v2 import Resize, Compose, RandomHorizontalFlip, ColorJitter, RandomAffine, RandomErasing, ToTensor

/u/x/i/xizheng/miniconda3/envs/inr/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/u/x/i/xizheng/miniconda3/envs/inr/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit 

In [ ]:
BASEDIR = '../rsna-2023-abdominal-trauma-detection'

TRAIN_IMG_PATH = os.path.join("../rsna-2023-png/", 'train_images')
TRAIN_META_PATH = os.path.join(BASEDIR, 'train_series_meta.csv')
TEST_IMG_PATH = os.path.join(BASEDIR, 'test_images')
TEST_META_PATH = os.path.join(BASEDIR, 'test_series_meta.csv')

TRAIN_LABEL_PATH = os.path.join(BASEDIR, 'train.csv')

In [3]:
def fetch_img_paths(train_img_path):
    img_paths = []
    
    print('Scanning directories...')
    for patient in tqdm(os.listdir(train_img_path)):
        for scan in os.listdir(os.path.join(TRAIN_IMG_PATH, patient)):
            scans = []
            for img in os.listdir(os.path.join(TRAIN_IMG_PATH, patient, scan)):
                scans.append(os.path.join(TRAIN_IMG_PATH, patient, scan, img))
            
            img_paths.append(scans)
            
    return img_paths

In [4]:
def select_elements_with_spacing(input_list, spacing):
    
    """
    Selects elements with a specified spacing from a given list.

    Args:
        input_list (list): The input list from which elements will be selected.
        spacing (int): The spacing between selected elements.

    Returns:
        list: A list of selected elements from the input list.

    Raises:
        ValueError: If the input list does not contain at least 4 * spacing elements.
    """
 
    if len(input_list) < spacing * 4:
        raise ValueError("List should contain at least 4 * spacing elements.")
        
        
    # We want to select elements in the middle part of the abdomen
    lower_bound = int(len(input_list) * 0.4)
    upper_bound = int(len(input_list) * 0.6)

    spacing = (upper_bound - lower_bound) // 3
    
    # start_index = random.randint(lower_bound, upper_bound)
    
    selected_indices = [lower_bound, lower_bound + spacing, lower_bound + (2*spacing), upper_bound]
    
    selected_elements = [input_list[index] for index in selected_indices]
    
    return selected_elements

In [15]:
def standardize_pixel_array(f):
    """
    Standardizes a DICOM pixel array by applying various transformations.
    
    Args:
        dicom_path (str): Path to the DICOM image file.
        
    Returns:
        np.ndarray: The standardized pixel array of the DICOM image.
    """
    dicom_image = pydicom.read_file(f)
    pixel_array = dicom_image.pixel_array
    
    if dicom_image.PixelRepresentation == 1:
        bit_shift = dicom_image.BitsAllocated - dicom_image.BitsStored
        dtype = pixel_array.dtype 
        new_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
        pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dicom_image)

    if dicom_image.PhotometricInterpretation == "MONOCHROME1":
        pixel_array = 1 - pixel_array

    # transform to hounsfield units
    intercept = dicom_image.RescaleIntercept
    slope = dicom_image.RescaleSlope
    pixel_array = pixel_array * slope + intercept

    # windowing
    window_center = int(dicom_image.WindowCenter)
    window_width = int(dicom_image.WindowWidth)
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    pixel_array = pixel_array.copy()
    pixel_array[pixel_array < img_min] = img_min
    pixel_array[pixel_array > img_max] = img_max

    # normalization
    if pixel_array.max() == pixel_array.min():
        pixel_array = np.zeros_like(pixel_array)  # Handle case of constant array
    else:
        pixel_array = (pixel_array - pixel_array.min()) / (pixel_array.max() - pixel_array.min())

    return pixel_array

def preprocess_jpeg(jpeg_path):
    img = cv2.imread(jpeg_path)
    greyscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)/255
    return greyscale

In [16]:
# dataset
class AbdominalData(Dataset):
    """
    Custom dataset class for handling abdominal trauma data classification.
    
    Args:
        df_path (str): Path to the CSV file containing patient labels.
        current_fold (int): The current fold for cross-validation.
        num_fold (int, optional): Total number of folds for cross-validation. Default is 5.
    """
    
    def __init__(self, df_path, current_fold, num_fold = 5):
        
        super().__init__()
        
        # collect all the image instance paths
        self.img_paths = fetch_img_paths(TRAIN_IMG_PATH)
                
        self.df = pd.read_csv(df_path)
        
        self.num_fold = num_fold
        self.current_fold = current_fold
        self.kf = KFold(n_splits=num_fold)
        
        self.transform = Compose([
                            RandomHorizontalFlip(),  # Randomly flip images left-right
                            ColorJitter(brightness=0.2),  # Randomly adjust brightness
                            ColorJitter(contrast=0.2),  # Randomly adjust contrast
                            RandomAffine(degrees=0, shear=10),  # Apply shear transformation
                            RandomAffine(degrees=0, scale=(0.8, 1.2)),  # Apply zoom transformation
                            RandomErasing(p=0.2, scale=(0.02, 0.2)), # Coarse dropout
                            ToTensor(),
                        ])
    
    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """
        
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        """
        Retrieves a sample from the dataset by index.
        
        Args:
            idx (int): Index of the dataset to retrieve.
        
        Returns:
            dict: A dictionary containing the image data and labels for different abdominal structures.
        """
        
        # sample 4 image instances
        dicom_images = select_elements_with_spacing(self.img_paths[idx],
                                                    spacing = 2)
        patient_id = dicom_images[0].split('/')[-3]
        images = []
        
        for d in dicom_images:
            image = preprocess_jpeg(d)
            images.append(image)
            
        images = np.stack(images)
        image = torch.tensor(images, dtype = torch.float).unsqueeze(dim = 1)
        
        image = self.transform(image).squeeze(dim = 1)
        
        label = self.df[self.df.patient_id == int(patient_id)].values[0][1:-1]
        
        # labels
        bowel = np.argmax(label[0:2], keepdims = True)
        extravasation = np.argmax(label[2:4], keepdims = True)
        kidney = np.argmax(label[4:7], keepdims = False)
        liver = np.argmax(label[7:10], keepdims = False)
        spleen = np.argmax(label[10:], keepdims = False)
        
        
        return {
            'image': image,
            'bowel': bowel,
            'extravasation': extravasation,
            'kidney': kidney,
            'liver': liver,
            'spleen': spleen,
        }
    
    def get_splits(self):
        fold_data = list(self.kf.split(self.img_paths))
        train_indices, val_indices = fold_data[self.current_fold]

        train_data = self._get_subset(train_indices)
        val_data = self._get_subset(val_indices)
        
        return train_data, val_data

    def _get_subset(self, indices):
        return Subset(self, indices)

In [7]:
class MetricsCalculator:
    
    def __init__(self, mode = 'binary'):
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
        
        self.mode = mode
    
    def update(self, logits, target):
        """
        Update the metrics calculator with predicted values and corresponding targets.
        
        Args:
            predicted (torch.Tensor): Predicted values.
            target (torch.Tensor): Ground truth targets.
        """
        if self.mode == 'binary':
            probabilities = torch.sigmoid(logits)
            predicted = (probabilities > 0.5)
        else:
            probabilities = F.softmax(logits, dim = 1)
            predicted = torch.argmax(probabilities, dim=1)
            
        self.probabilities.extend(probabilities.detach().cpu().numpy())
        self.predictions.extend(predicted.detach().cpu().numpy())
        self.targets.extend(target.detach().cpu().numpy())
    
    def reset(self):
        """Reset the stored predictions and targets."""
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
    
    def compute_accuracy(self):
        return accuracy_score(self.targets, self.predictions)
    
    def compute_auc(self):
        """
        Compute the AUC (Area Under the Curve) metric.
        
        Returns:
            float: AUC.
        """
        if self.mode == 'multi':
            return roc_auc_score(self.targets, self.probabilities, multi_class = 'ovo', labels=[0, 1, 2])
    
        else:
            return roc_auc_score(self.targets, self.probabilities)

In [8]:
# Model Architecure
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.input = nn.Conv2d(4, 3, kernel_size = 3)
        model = models.efficientnet_b0(weights = 'IMAGENET1K_V1')
        
        self.features = model.features
        self.avgpool = model.avgpool
        
        self.bowel = nn.Linear(1280, 1)
        self.extravasation = nn.Linear(1280, 1)
        self.kidney = nn.Linear(1280, 3)
        self.liver = nn.Linear(1280,3) 
        self.spleen = nn.Linear(1280, 3)
    
    def forward(self, x):
        
        # extract features
        x = self.input(x)
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        
        # output logits
        bowel = self.bowel(x)
        extravsation = self.extravasation(x)
        kidney = self.kidney(x)
        liver = self.liver(x)
        spleen = self.spleen(x)
        
        return bowel, extravsation, kidney, liver, spleen

In [9]:

model = CNNModel().to('cuda')

In [10]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
LR = 1e-4

In [11]:
train_data_0, val_data_0 = AbdominalData(TRAIN_LABEL_PATH, current_fold=0).get_splits()
train_data_1, val_data_1 = AbdominalData(TRAIN_LABEL_PATH, current_fold=1).get_splits()
train_data_2, val_data_2 = AbdominalData(TRAIN_LABEL_PATH, current_fold=2).get_splits()
train_data_3, val_data_3 = AbdominalData(TRAIN_LABEL_PATH, current_fold=3).get_splits()
train_data_4, val_data_4 = AbdominalData(TRAIN_LABEL_PATH, current_fold=4).get_splits()

train_dataloader_0 = DataLoader(train_data_0,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader_0 = DataLoader(val_data_0,batch_size = BATCH_SIZE, shuffle = False)
train_dataloader_1 = DataLoader(train_data_1,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader_1 = DataLoader(val_data_1,batch_size = BATCH_SIZE, shuffle = False)
train_dataloader_2 = DataLoader(train_data_2,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader_2 = DataLoader(val_data_2,batch_size = BATCH_SIZE, shuffle = False)
train_dataloader_3 = DataLoader(train_data_3,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader_3 = DataLoader(val_data_3,batch_size = BATCH_SIZE, shuffle = False)
train_dataloader_4 = DataLoader(train_data_4,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader_4 = DataLoader(val_data_4,batch_size = BATCH_SIZE, shuffle = False)

Scanning directories...


  0%|          | 0/3147 [00:00<?, ?it/s]

Scanning directories...


/u/x/i/xizheng/miniconda3/envs/inr/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


  0%|          | 0/3147 [00:00<?, ?it/s]

...

  0%|          | 0/3147 [00:00<?, ?it/s]

Scanning directories...


/u/x/i/xizheng/miniconda3/envs/inr/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


  0%|          | 0/3147 [00:00<?, ?it/s]

/u/x/i/xizheng/miniconda3/envs/inr/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr = LR)
bce_b = nn.BCEWithLogitsLoss(pos_weight = torch.tensor([2.0]).to('cuda'))
bce_e = nn.BCEWithLogitsLoss(pos_weight = torch.tensor([4.0]).to('cuda'))
cce = nn.CrossEntropyLoss(label_smoothing = 0.05, weight = torch.tensor([1.0, 2.0, 4.0]).to('cuda'))
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)

In [13]:
# initialize metrics objects
train_acc_bowel = MetricsCalculator('binary')
train_acc_extravasation = MetricsCalculator('binary')
train_acc_liver = MetricsCalculator('multi')
train_acc_kidney = MetricsCalculator('multi')
train_acc_spleen = MetricsCalculator('multi')

val_acc_bowel = MetricsCalculator('binary')
val_acc_extravasation = MetricsCalculator('binary')
val_acc_liver = MetricsCalculator('multi')
val_acc_kidney = MetricsCalculator('multi')
val_acc_spleen = MetricsCalculator('multi')

In [14]:
prev_val_best_loss = float('inf')

dataloaders = [(train_dataloader_0, val_dataloader_0),
               (train_dataloader_1, val_dataloader_1),
               (train_dataloader_2, val_dataloader_2), 
               (train_dataloader_3, val_dataloader_3),
               (train_dataloader_4, val_dataloader_4)]

for epoch in range(NUM_EPOCHS):
    
    # training
    model.train()
    
    train_loss = 0.0
    val_loss = 0.0
    
    print(f'Epoch: [{epoch+1}/{NUM_EPOCHS}]')
    
    train_dataloader, val_dataloader = dataloaders[epoch%5]
    
    print(f'Fold: {epoch%5}')
    
    for batch_idx, batch_data in enumerate(tqdm(train_dataloader)):
        print(batch_data)
        inputs = batch_data['image'].to('cuda')
        bowel = batch_data['bowel'].to('cuda')
        extravasation = batch_data['extravasation'].to('cuda')
        liver = batch_data['liver'].to('cuda')
        kidney = batch_data['kidney'].to('cuda')
        spleen = batch_data['spleen'].to('cuda')
        
        optimizer.zero_grad()
        b, e, k, l, s = model(inputs)
        b_loss = bce_b(b, bowel.float())
        e_loss = bce_e(e, extravasation.float())
        l_loss = cce(l, liver)
        k_loss = cce(k, kidney)
        s_loss = cce(s, spleen)
        
        total_loss = b_loss + e_loss + l_loss + k_loss + s_loss
        total_loss.backward()
        
        optimizer.step()
        
        # calculate training metrics
        train_loss += total_loss.item()
        train_acc_bowel.update(b, bowel)
        train_acc_extravasation.update(e, extravasation)
        train_acc_liver.update(l, liver)
        train_acc_kidney.update(k, kidney)
        train_acc_spleen.update(s, spleen)
    
    train_loss = train_loss/(batch_idx+1)
    
    # validation
    model.eval()
    running_loss = 0.0
    
    for batch_idx, batch_data in enumerate(tqdm(val_dataloader)):
        inputs = batch_data['image'].to('cuda')
        bowel = batch_data['bowel'].to('cuda')
        extravasation = batch_data['extravasation'].to('cuda')
        liver = batch_data['liver'].to('cuda')
        kidney = batch_data['kidney'].to('cuda')
        spleen = batch_data['spleen'].to('cuda')

        
        b, e, k, l, s = model(inputs)
        b_loss = bce_b(b, bowel.float())
        e_loss = bce_e(e, extravasation.float())
        l_loss = cce(l, liver)
        k_loss = cce(k, kidney)
        s_loss = cce(s, spleen)
        
        total_loss = b_loss + e_loss + l_loss + k_loss + s_loss
        
        # calculate validation metrics
        val_loss += total_loss.item()
        val_acc_bowel.update(b, bowel)
        val_acc_extravasation.update(e, extravasation)
        val_acc_liver.update(l, liver)
        val_acc_kidney.update(k, kidney)
        val_acc_spleen.update(s, spleen)
    
    
    val_loss = val_loss/(batch_idx+1)
    scheduler.step(val_loss)
    
    if val_loss < prev_val_best_loss:
        prev_val_best_loss = val_loss
        print("Validation Loss improved, Saving Model...")
        torch.save(model, f'efficientnet_b0_{val_loss:.3f}.pth')
    
    
    
    # accuracy and auc data
    metrics_data = [
                    ["Bowel", 
                        train_acc_bowel.compute_accuracy(),
                        val_acc_bowel.compute_accuracy(),
                        train_acc_bowel.compute_auc(),
                        val_acc_bowel.compute_auc()],
                    ["Extravasation", 
                        train_acc_extravasation.compute_accuracy(),
                        val_acc_extravasation.compute_accuracy(),
                        train_acc_extravasation.compute_auc(),
                        val_acc_extravasation.compute_auc()],
                    ["Liver", 
                        train_acc_liver.compute_accuracy(),
                        val_acc_liver.compute_accuracy(),
                        train_acc_liver.compute_auc(),
                        val_acc_liver.compute_auc()],
                    ["Kidney", 
                        train_acc_kidney.compute_accuracy(),
                        val_acc_kidney.compute_accuracy(),
                        train_acc_kidney.compute_auc(),
                        val_acc_kidney.compute_auc()],
                    ["Spleen", 
                        train_acc_spleen.compute_accuracy(),
                        val_acc_spleen.compute_accuracy(),
                        train_acc_spleen.compute_auc(),
                        val_acc_spleen.compute_auc()]
                ]
    
    # verbose
    print('')
    print(tabulate(metrics_data, headers=["", "Train Acc", "Val Acc", "Train AUC", "Val AUC"]))
    
    print(f'\nMean Train Loss: {train_loss:.3f}')
    print(f'Mean Val Loss: {val_loss:.3f}\n')
    
    #reset metrics
    train_acc_bowel.reset()
    train_acc_extravasation.reset()
    train_acc_liver.reset()
    train_acc_kidney.reset()
    train_acc_spleen.reset()
    val_acc_bowel.reset()
    val_acc_extravasation.reset()
    val_acc_liver.reset()
    val_acc_kidney.reset()
    val_acc_spleen.reset()


Epoch: [1/10]
Fold: 0


  0%|          | 0/236 [00:00<?, ?it/s]

RuntimeError: stack expects each tensor to be equal size, but got [4, 512, 512] at entry 0 and [4, 512, 761] at entry 1